In [1]:
import cv2
import numpy as np
import os

def enhance_plant_region(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([35, 60, 60])
    upper_green = np.array([85, 255, 255])
    plant_mask = cv2.inRange(hsv, lower_green, upper_green)
    kernel = np.ones((5, 5), np.uint8)
    plant_mask = cv2.morphologyEx(plant_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    plant_mask = cv2.morphologyEx(plant_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    return plant_mask

def analyze_image(image_path, day, photo_num, show_preview=False):
    image = cv2.imread(image_path)
    if image is None:
        return "Error: Could not load image."

    image = cv2.resize(image, (600, 400))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    plant_mask = enhance_plant_region(image)

    # Yellow color range
    lower_yellow = np.array([20, 120, 120])
    upper_yellow = np.array([40, 255, 255])
    yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
    yellow_in_plant = cv2.bitwise_and(yellow_mask, yellow_mask, mask=plant_mask)

    yellow_pixels = cv2.countNonZero(yellow_in_plant)
    plant_pixels = cv2.countNonZero(plant_mask)
    yellow_ratio = yellow_pixels / plant_pixels if plant_pixels > 0 else 0

    # Optional preview
    if show_preview:
        contours, _ = cv2.findContours(yellow_in_plant, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        result_img = image.copy()
        cv2.drawContours(result_img, contours, -1, (0, 0, 255), 2)
        cv2.putText(result_img, f'Yellowing: {yellow_ratio*100:.2f}%', 
                    (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow(f"Day {day} - Photo {photo_num}", result_img)
        cv2.waitKey(300)
        cv2.destroyAllWindows()

    # Status message
    if yellow_ratio > 0.07:
        return f"Warning: {yellow_ratio*100:.2f}% yellowing detected. Possible disease."
    else:
        return f"Healthy: {yellow_ratio*100:.2f}% yellowing detected."

# === Parameters ===
days = 14
photos_per_day = 4
base_dir = r"C:\Users\DELL\Desktop\IP Project"  # Update if your path is different
txt_filename = "plant_growth_report.txt"
show_images = False  # Set to True to enable previews

# === Processing ===
data = {}

for day in range(1, days + 1):
    data[f"Day {day}"] = {}
    for photo_num in range(1, photos_per_day + 1):
        image_filename = f"DAY-{day} ({photo_num}).jpg"
        image_path = os.path.join(base_dir, image_filename)
        print(f"Analyzing: {image_path}")
        message = analyze_image(image_path, day, photo_num, show_preview=show_images)
        data[f"Day {day}"][f"Photo {photo_num}"] = message
        print(f"Day {day} - Photo {photo_num}: {message}")

# === Write Report ===
with open(txt_filename, "w", encoding='utf-8') as file:
    for day, photos in data.items():
        file.write(f"{day}\n")
        for photo, msg in photos.items():
            file.write(f"  {photo}: {msg}\n")
        file.write("\n")

print(f"\n✅ Analysis complete. Report saved as: {txt_filename}")

Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-1 (1).jpg
Day 1 - Photo 1: Warning: 29.08% yellowing detected. Possible disease.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-1 (2).jpg
Day 1 - Photo 2: Healthy: 3.75% yellowing detected.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-1 (3).jpg
Day 1 - Photo 3: Healthy: 0.36% yellowing detected.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-1 (4).jpg
Day 1 - Photo 4: Healthy: 0.57% yellowing detected.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-2 (1).jpg
Day 2 - Photo 1: Warning: 29.08% yellowing detected. Possible disease.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-2 (2).jpg
Day 2 - Photo 2: Healthy: 3.75% yellowing detected.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-2 (3).jpg
Day 2 - Photo 3: Healthy: 0.36% yellowing detected.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-2 (4).jpg
Day 2 - Photo 4: Healthy: 0.57% yellowing detected.
Analyzing: C:\Users\DELL\Desktop\IP Project\DAY-3 (1).jpg
Day 3 - Photo 1: Healthy